In [13]:
import os, tqdm, json, pickle, gc, zipfile, itertools, time, collections
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from datetime import datetime
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
import os, tqdm, json, pickle, gc, zipfile, itertools, time, collections
import pandas as pd
import numpy as np
from dateutil import parser
import warnings
warnings.filterwarnings('ignore')
from collections import defaultdict
from multiprocessing import Pool
import catboost as cb
from sklearn.model_selection import train_test_split, TimeSeriesSplit, cross_val_score, ParameterGrid, StratifiedKFold, cross_val_predict
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance
import optuna
from optuna.samplers import TPESampler
from tqdm.contrib.concurrent import process_map  
import seaborn as sns
import matplotlib.pyplot as plt
import shap 
from sklearn.model_selection import KFold
from nancorrmp.nancorrmp import NaNCorrMp
# from pathos.multiprocessing import ProcessingPool as Pool
import multiprocessing as mp
from datetime import datetime
from optuna.samplers import TPESampler

def objective(trial):

    from sklearn.metrics import f1_score

    param = {
        "objective": trial.suggest_categorical("objective", ["Logloss"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", .5, 1),
        "depth": trial.suggest_int("depth", 3, 11),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli"]
        )
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    param.update(CONST_PARAMS)
    param['verbose'] = 0
    param['cat_features'] = np.where(X_train.dtypes=='category')[0]
    param['use_best_model'] = True
    param['random_state'] = SEED    

    model = cb.CatBoostClassifier(**param)
    model.fit(X_train, y_train,  eval_set=(X_valid, y_valid), early_stopping_rounds=EARLY_STOPPING)

    score = roc_auc_score(y_valid, model.predict_proba(X_valid)[:, 1])    
    
    return score


from sklearn.base import BaseEstimator, TransformerMixin

class BacwardFeatureSelector(BaseEstimator, TransformerMixin):

    def __init__(self, CONST_PARAMS, SEED, N_PERM_ITER, EARLY_STOPPING, SCORING, scoring_func, proba):

        self.CONST_PARAMS = CONST_PARAMS
        self.SEED = SEED
        self.N_PERM_ITER = N_PERM_ITER
        self.EARLY_STOPPING = EARLY_STOPPING
        self.SCORING = SCORING
        self.scoring_func = scoring_func
        self.proba = proba

    def fit(self, X_tr, y_tr,X_val, y_val):

        X_tr_c, X_val_c = X_tr.copy(), X_val.copy()

        I = 1
        # рекурсивно отбираем признаки с помощью ранней остановки
        while True:

            print('> step#1, iter#{}, selected features counter: {}'.format(I, X_tr_c.shape[1]))

            params = self.CONST_PARAMS.copy()
            params['cat_features'] = np.where(X_tr_c.dtypes=='category')[0]
            params['use_best_model'] = True
            params['random_state'] = self.SEED

            model = cb.CatBoostClassifier(**params)   
            model.fit(X_tr_c, y_tr,  eval_set=(X_val_c, y_val), early_stopping_rounds=self.EARLY_STOPPING)

            mask = model.feature_importances_>0

            if np.all(mask):
                break
            else:
                X_tr_c, X_val_c = X_tr_c.loc[:, mask], X_val_c.loc[:, mask]
                I+=1

        # рекурсивно отбираем признаки с помощью изменения метрики при перемешивании строк
        I=1        
        while True:

            print('> step#2, iter#{}, selected features counter: {}'.format(I, X_tr_c.shape[1]))

            params = self.CONST_PARAMS.copy()
            params['cat_features'] = np.where(X_tr_c.dtypes=='category')[0]
            params['use_best_model'] = True
            params['random_state'] = SEED
            params['verbose'] = 0

            model = cb.CatBoostClassifier(**params)   
            model.fit(X_tr_c, y_tr,  eval_set=(X_val_c, y_val), early_stopping_rounds=self.EARLY_STOPPING)  
            
            if self.proba:
                score_before= self.scoring_func(y_val, model.predict_proba(X_val_c)[:,1])
            else:
                score_before= self.scoring_func(y_val, model.predict(X_val_c))

            L_perm_imp = []
            for i in tqdm.tqdm(range(N_PERM_ITER)):
                arr_perm_imp = permutation_importance(model, X_val_c, y_val,n_repeats=1, scoring = self.SCORING, random_state= self.SEED+i, n_jobs=-1)['importances_mean']
                L_perm_imp.append(arr_perm_imp)
            
            mask = np.r_[L_perm_imp].mean(0)>0

            if np.all(mask):
                break
            else:
                if self.proba:                    
                    params['cat_features'] = np.where(X_tr_c.loc[:, mask].dtypes=='category')[0]
                    model = cb.CatBoostClassifier(**params)   
                    model.fit(X_tr_c.loc[:, mask], y_tr,  eval_set=(X_val_c.loc[:, mask], y_val), early_stopping_rounds=self.EARLY_STOPPING) 
                    score_after= self.scoring_func(y_val, model.predict_proba(X_val_c.loc[:, mask])[:,1])
                else:
                    params['cat_features'] = np.where(X_tr_c.loc[:, mask].dtypes=='category')[0]
                    model = cb.CatBoostClassifier(**params)   
                    model.fit(X_tr_c.loc[:, mask], y_tr,  eval_set=(X_val_c.loc[:, mask], y_val), early_stopping_rounds=self.EARLY_STOPPING) 
                    score_after= self.scoring_func(y_val, model.predict(X_val_c.loc[:, mask]))
                if score_after>score_before:    
                    X_tr_c, X_val_c = X_tr_c.loc[:, mask], X_val_c.loc[:, mask]
                    I+=1
                else:
                    break
        
        self.best_features= X_tr_c.columns
        del X_tr_c, X_val_c
        gc.collect()

        return self

    def transform(self, X_te):
        return X_te[self.best_features]

from sklearn.metrics import f1_score
def select_features(X_train, X_valid, X_test, y_train, y_valid, y_test):

    n_batches = np.int32(np.ceil(X_train.shape[1]/1000))
    L_FEATURE_BATCH=np.array_split(X_train.columns, n_batches)

    L_selected_features = []
    for i, fb in enumerate(L_FEATURE_BATCH):

        print('ITER#{}/{}'.format(i+1, n_batches))    

        X_batch_train, X_batch_valid = X_train[fb].copy(), X_valid[fb].copy()

        params = CONST_PARAMS.copy()
        params['cat_features'] = np.where(X_batch_train.dtypes=='category')[0]
        params['use_best_model'] = True
        params['random_state'] = SEED

        model = cb.CatBoostClassifier(**params)   
        model.fit(X_batch_train, y_train,  eval_set=(X_batch_valid, y_valid), early_stopping_rounds=EARLY_STOPPING)

        mask = model.feature_importances_>0     

        L_selected_features.extend(fb[mask].tolist())

        del X_batch_train, X_batch_valid
        gc.collect()

    X_train_c, X_valid_c, X_test_c = X_train[L_selected_features], X_valid[L_selected_features], X_test[L_selected_features]
    del X_train, X_valid, X_test
    X_train, X_valid, X_test = X_train_c.copy(), X_valid_c.copy(), X_test_c.copy()
    del X_train_c, X_valid_c, X_test_c
    gc.collect()

    bfs = BacwardFeatureSelector(
        CONST_PARAMS,
        SEED,
        N_PERM_ITER,
        EARLY_STOPPING,
        SCORING,
        proba= True,
        scoring_func=lambda y, y_proba: roc_auc_score(y, y_proba)
    )

    bfs.fit(X_train, y_train, X_valid, y_valid)

    X_train_c = bfs.transform(X_train)
    X_valid_c = bfs.transform(X_valid)
    X_test_c = bfs.transform(X_test)
    del X_train, X_valid, X_test
    X_train, X_valid, X_test = X_train_c.copy(), X_valid_c.copy(), X_test_c.copy()
    del X_train_c, X_valid_c, X_test_c
    gc.collect()

    from sklearn.metrics import accuracy_score
    # study = optuna.create_study(direction="maximize")
    # study.optimize(objective, n_trials=100, timeout=600)
    # best_params=study.best_trial.params
    params = CONST_PARAMS.copy()
    params['cat_features'] = np.where(X_train.dtypes=='category')[0]
    params['use_best_model'] = True
    params['random_state'] = SEED

    model = cb.CatBoostClassifier(**params)      
    model.fit(X_train, y_train,  eval_set=(X_valid, y_valid), early_stopping_rounds=EARLY_STOPPING)
    te_proba = model.predict_proba(X_test)[:, 1]
    score = 

    d_results = {} 
    d_results['features'] = X_test.columns
    d_results['roc_auc'] = roc_auc_score(y_test, te_proba)
    d_results['accuracy'] = accuracy_score(y_test, np.int32(np.rint(te_proba)))
    d_results['proba'] = te_proba
    
    return d_results

def get_responses(PATH_TO_RESPONSES):
    L_FILENAMES = os.listdir(PATH_TO_RESPONSES)
    L_RESPONSES = []
    for fnm in tqdm.tqdm(L_FILENAMES):
        try:
            pth = os.path.join(PATH_TO_RESPONSES, fnm)
            with open(pth, 'r') as f:
                d_rsp = json.load(f)
            L_RESPONSES.append(d_rsp)
        except:
            pass
    idx_ordered = np.argsort([d_game['id'] for d_game in L_RESPONSES])[::-1]
    L_RESPONSES = np.array(L_RESPONSES)[idx_ordered].tolist()
    return L_RESPONSES

def get_game_profile(L_RESPONSES):

    L_STAT_KEYS=[
        'adr', 'assists', 'deaths', 'first_kills_diff', 'flash_assists',
        'headshots', 'k_d_diff', 'kast', 'kills', 'rating'
    ]

    L_games =[]
    for d_rsp in tqdm.tqdm(L_RESPONSES):

        try:
            d_game= {}

            d_game['id']= d_rsp['id']

            date = parser.parse(d_rsp['begin_at'])
            d_game['timestamp'] = date.timestamp()
            d_game['year'] = date.year
            d_game['month'] = date.month
            d_game['day'] = date.day
            d_game['weekday'] = date.weekday()
            d_game['hour'] = date.hour

            d_game['map_id'] = d_rsp['map']['id']

            d_game['league_id'] = d_rsp['match']['league_id']
            d_game['serie_id'] = d_rsp['match']['serie_id']
            d_game['tournament_id'] = d_rsp['match']['tournament_id']
            d_game['serie_tier'] = d_rsp['match']['serie']['tier']
            d_game['tournament_name'] = str.lower(d_rsp['match']['tournament']['name'])
            pp = d_rsp['match']['tournament']['prizepool']
            try:
                pp_size = int(pp.split(' ')[0])
                pp_curr = str.lower(''.join(pp.split(' ')[1:]))
            except:
                pp_size = np.nan
                pp_curr = np.nan
            d_game['tournament_pp_size'] = pp_size
            d_game['tournament_pp_curr'] = pp_curr




            df_rounds = pd.DataFrame.from_records(d_rsp['rounds']) 
            d_h1_win_count = df_rounds.query('round<=15')['winner_team'].value_counts().to_dict()
            d_h2_win_count = df_rounds.query('round>15')['winner_team'].value_counts().to_dict()
            d_h1_outcome_count = df_rounds.query('round<=15')['outcome'].value_counts().to_dict()
            d_h2_outcome_count = df_rounds.query('round>15')['outcome'].value_counts().to_dict()

            d_game['team1_id']=df_rounds.query('round==1')['ct'].iloc[0]   
            d_game['team2_id']=df_rounds.query('round==1')['terrorists'].iloc[0]            

            d_game['team1_maxround'] = df_rounds['round'].max()
            d_game['team1_win'] = int(df_rounds['winner_team'].value_counts().idxmax() == d_game['team1_id'])
            
            d_game['team1_h1_win_count'] = d_h1_win_count[d_game['team1_id']] if d_game['team1_id'] in d_h1_win_count.keys() else 0.0
            d_game['team1_h2_win_count'] = d_h2_win_count[d_game['team1_id']] if d_game['team1_id'] in d_h2_win_count.keys() else 0.0
            d_game['team2_h1_win_count'] = d_h1_win_count[d_game['team2_id']] if d_game['team2_id'] in d_h1_win_count.keys() else 0.0
            d_game['team2_h2_win_count'] = d_h2_win_count[d_game['team2_id']] if d_game['team2_id'] in d_h2_win_count.keys() else 0.0
            

            for half, d in zip(['h1', 'h2'], [d_h1_outcome_count, d_h2_outcome_count]):
                for outcome, count in d.items():
                    d_game[f'team1_{half}_{outcome}_count']=count
                    

            

            if len(d_rsp['players'])==10:

                team1_counter, team2_counter = 1, 6
                for p in d_rsp['players']:

                    team_id = p['team']['id']

                    if team_id==d_game['team1_id']:
                        
                        d_game['team1_location'] = p['team']['location']
                        
                        d_game[f'player{team1_counter}_id'] = p['player']['id']
                        d_game[f'player{team1_counter}_nationality'] = p['player']['nationality']
                        d_game[f'player{team1_counter}_hometown'] = p['player']['hometown']
                        d_game[f'player{team1_counter}_age'] = p['player']['age']

                        for key in L_STAT_KEYS:
                            value= p[key]
                            d_game[f'player{team1_counter}_{key}'] = value if pd.notnull(value) else 0.0
                        

                        team1_counter+=1

                    else:
                        
                        d_game['team2_location'] = p['team']['location']

                        d_game[f'player{team2_counter}_id'] = p['player']['id']
                        d_game[f'player{team2_counter}_nationality'] = p['player']['nationality']
                        d_game[f'player{team2_counter}_hometown'] = p['player']['hometown']
                        d_game[f'player{team2_counter}_age'] = p['player']['age']

                        for key in L_STAT_KEYS:
                            value= p[key]
                            d_game[f'player{team2_counter}_{key}'] = value if pd.notnull(value) else 0.0
                        

                        team2_counter+=1


                L_games.append(d_game)
                del d_game

        except:
            pass

    df_game_profile = pd.DataFrame.from_records(L_games)
    del L_games
    gc.collect()

    L_PLAYERS4TEAM1 = ['player1_id', 'player2_id', 'player3_id', 'player4_id', 'player5_id']
    L_PLAYERS4TEAM2 = ['player6_id', 'player7_id', 'player8_id', 'player9_id', 'player10_id']

    df_game_profile['team1_lineup'] = df_game_profile[L_PLAYERS4TEAM1].apply(lambda x: '-'.join(x.sort_values().astype(str)), axis = 1)
    df_game_profile['team2_lineup'] = df_game_profile[L_PLAYERS4TEAM2].apply(lambda x: '-'.join(x.sort_values().astype(str)), axis = 1)

    return df_game_profile.sort_values('timestamp').reset_index(drop = True)

def factorize(df_game_profile, to_enc):

    df_game_profile_c = df_game_profile.copy()
    del df_game_profile
    gc.collect()

    unique_values = pd.Series(df_game_profile_c[to_enc].values.flatten()).unique()
    labels = range(len(unique_values))
    d = dict(zip(unique_values, labels))
    df_game_profile_c[to_enc] = df_game_profile_c[to_enc].applymap(lambda x: d[x] if x in d.keys() else FILL_VALUE)

    return df_game_profile_c

def scale(x, out_range=(-1, 1)):
    domain = np.min(x), np.max(x)
    y = (x - (domain[1] + domain[0]) / 2) / (domain[1] - domain[0])
    return y * (out_range[1] - out_range[0]) + (out_range[1] + out_range[0]) / 2


def get_game_profile(L_RESPONSES):

    L_STAT_KEYS=[
        'adr', 'assists', 'deaths', 'first_kills_diff', 'flash_assists',
        'headshots', 'k_d_diff', 'kast', 'kills', 'rating'
    ]

    L_games =[]
    for d_rsp in tqdm.tqdm(L_RESPONSES):

        try:
            d_game= {}

            d_game['id']= d_rsp['id']

            date = parser.parse(d_rsp['begin_at'])
            d_game['timestamp'] = date.timestamp()
            d_game['year'] = date.year
            d_game['month'] = date.month
            d_game['day'] = date.day
            d_game['weekday'] = date.weekday()
            d_game['hour'] = date.hour

            d_game['map_id'] = d_rsp['map']['id']

            d_game['league_id'] = d_rsp['match']['league_id']
            d_game['serie_id'] = d_rsp['match']['serie_id']
            d_game['tournament_id'] = d_rsp['match']['tournament_id']
            d_game['serie_tier'] = d_rsp['match']['serie']['tier']
            d_game['tournament_name'] = str.lower(d_rsp['match']['tournament']['name'])
            pp = d_rsp['match']['tournament']['prizepool']
            try:
                pp_size = int(pp.split(' ')[0])
                pp_curr = str.lower(''.join(pp.split(' ')[1:]))
            except:
                pp_size = np.nan
                pp_curr = np.nan
            d_game['tournament_pp_size'] = pp_size
            d_game['tournament_pp_curr'] = pp_curr


            df_rounds = pd.DataFrame.from_records(d_rsp['rounds']) 
            d_h1_win_count = df_rounds.query('round<=15')['winner_team'].value_counts().to_dict()
            d_h2_win_count = df_rounds.query('round>15')['winner_team'].value_counts().to_dict()
            d_h1_outcome_count = df_rounds.query('round<=15')['outcome'].value_counts().to_dict()
            d_h2_outcome_count = df_rounds.query('round>15')['outcome'].value_counts().to_dict() 
            winner_id = df_rounds['winner_team'].value_counts().idxmax()
            maxround = df_rounds['round'].max()
            start_ct_id = df_rounds.query('round==1')['ct'].iloc[0]

            if len(d_rsp['players'])==10:
                
                for p in d_rsp['players']:
                    
                    d_game_c = d_game.copy()

                    d_game_c['team_id'] = p['team']['id']
                    d_game_c['opponent_id'] = p['opponent']['id']

                    d_game_c['team_location'] = p['team']['location']
                        
                    d_game_c['player_id'] = p['player']['id']
                    d_game_c['player_nationality'] = p['player']['nationality']
                    d_game_c['player_hometown'] = p['player']['hometown']
                    d_game_c['player_age'] = p['player']['age']

                    d_game_c['start_ct'] = 1 if d_game_c['team_id']==start_ct_id else 0
                    d_game_c['win'] = 1 if d_game_c['team_id']==winner_id else 0
                    d_game_c['maxround'] = maxround

                    d_game_c['h1_win_count'] = d_h1_win_count[d_game_c['team_id']] if d_game_c['team_id'] in d_h1_win_count.keys() else 0.0
                    d_game_c['h2_win_count'] = d_h2_win_count[d_game_c['team_id']] if d_game_c['team_id'] in d_h2_win_count.keys() else 0.0

                    for k, v in d_h1_outcome_count.items():
                        d_game_c[f'h1_{k}_count'] = v
                    for k, v in d_h2_outcome_count.items():
                        d_game_c[f'h2_{k}_count'] = v

                    for key in L_STAT_KEYS:
                        value= p[key]
                        d_game_c[key] = value if pd.notnull(value) else 0.0    

                    L_games.append(d_game_c)
                    del d_game_c

        except:
            pass

    df_games = pd.DataFrame.from_records(L_games)
    del L_games
    gc.collect()

    return df_games
    

def get_responses(PATH_TO_RESPONSES):
    L_FILENAMES = os.listdir(PATH_TO_RESPONSES)
    L_RESPONSES = []
    for fnm in tqdm.tqdm(L_FILENAMES):
        try:
            pth = os.path.join(PATH_TO_RESPONSES, fnm)
            with open(pth, 'r') as f:
                d_rsp = json.load(f)
            L_RESPONSES.append(d_rsp)
        except:
            pass
    idx_ordered = np.argsort([d_game['id'] for d_game in L_RESPONSES])[::-1]
    L_RESPONSES = np.array(L_RESPONSES)[idx_ordered].tolist()
    return L_RESPONSES

def reduce_mem_usage(series):
    try:
        col_type = series.dtype

        if col_type != object:
            c_min = series.min()
            c_max = series.max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    series = series.astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    series = series.astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    series = series.astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    series = series.astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    series = series.astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    series = series.astype(np.float32)
                else:
                    series = series.astype(np.float64)
        else:
            pass 
        return series
        
    except:
        pass

    

FILL_VALUE = -999
CONST_PARAMS= {
            'iterations':1000,
            'loss_function':'Logloss',    
            'verbose':1,
            }
SEED=13
N_PERM_ITER = 10
EARLY_STOPPING=20
SCORING= 'roc_auc'

SyntaxError: invalid syntax (<ipython-input-13-f1198eae83f1>, line 231)

### 1. респонсы

In [2]:
# # директория с респонсами
# PATH_TO_RESPONSES = 'L_games_collection'

# # респонсы
# L_RESPONSES = get_responses(PATH_TO_RESPONSES)

### 2. игры (стата команд+игроков)

In [3]:
# # профайлинг игр
# df_game_profile = get_game_profile(L_RESPONSES)

# df_game_profile['lineup'] = np.nan
# for (game_id, team_id), subdf in tqdm.tqdm(df_game_profile.groupby(['id', 'team_id'])):
#     lineup = '-'.join(subdf['player_id'].sort_values().astype(str))
#     df_game_profile.loc[subdf.index, 'lineup'] = lineup    

df_game_profile = pd.read_pickle('df_game_profile_21052022.pickle')

### 3. сплит 

In [4]:
# сплит
train_id = pd.read_csv('train_id.txt', index_col=0)['id'].values
test_id = pd.read_csv('test_id.txt', index_col=0)['id'].values

df_game_profile_train = df_game_profile.query('id in @train_id').reset_index(drop = True)
df_game_profile_test = df_game_profile.query('id in @test_id').reset_index(drop = True)

L_GAME_ID_TRAIN = np.unique(df_game_profile_train['id'])
L_GAME_ID_TEST = np.unique(df_game_profile_test['id'])

### 4. извлечение признаков

In [5]:
L_INFO_KEYS = [
    'id', 'timestamp', 'year', 'month', 'day', 'weekday', 'hour', 'map_id',
    'league_id', 'serie_id', 'tournament_id', 'serie_tier',
    'tournament_name', 'tournament_pp_size', 'tournament_pp_curr'
]

L_BY_KEYS = ['year', 'month', 'day','weekday','hour', 'serie_tier']

L_TEAM_AGG_KEYS = [    
    'win', 'maxround', 'h1_win_count', 'h2_win_count',
   
    'h1_eliminated_count', 'h1_defused_count', 'h1_exploded_count',
    'h2_eliminated_count', 'h2_defused_count', 'h2_exploded_count',
    'h2_timeout_count',
    'h1_timeout_count',
    
    'adr','assists', 'deaths', 'first_kills_diff', 'flash_assists', 'headshots',
    'k_d_diff', 'kast', 'kills', 'rating'
]

L_PLAYER_AGG_KEYS = [          
    'adr','assists', 'deaths', 'first_kills_diff', 'flash_assists', 'headshots',
    'k_d_diff', 'kast', 'kills', 'rating'
]  

In [10]:
PATH2DIR_TRAIN = r'D:\features_train_21052022'
train_in = [int(x.split('.')[0]) for x in os.listdir(PATH2DIR_TRAIN)]
train_new = list(set(L_GAME_ID_TRAIN) - set(train_in))
L_GAME_ID_TRAIN = train_new

for game_id in tqdm.tqdm(L_GAME_ID_TRAIN[::-1]):

    try:

        # путь до выходного файлла
        path_out = os.path.join(PATH2DIR_TRAIN, '{}.pickle'.format(int(game_id)))

        # игра
        df_game = df_game_profile_train.query('id==@game_id')

        # таймстемп игры
        timestamp = df_game['timestamp'].iloc[0]
        # карта игры
        map_id = df_game['map_id'].iloc[0]

        
        # признаки для игры
        d_fs4gm = df_game.iloc[0][L_INFO_KEYS].to_dict()

        d_team_id2start_ct = dict(zip(df_game['team_id'], df_game['start_ct']))
        d_team_id2opponent_id = dict(zip(df_game['team_id'], df_game['opponent_id']))
        d_team_id2lineup = dict(zip(df_game['team_id'], df_game['lineup']))
        d_start_ct2prefix= {1:'team1', 0:'team2'}

        # для каждой команды в игре
        for team_id, start_ct in d_team_id2start_ct.items():
            
            prefix = d_start_ct2prefix[start_ct]
            opponent_id = d_team_id2opponent_id[team_id]
            lineup = d_team_id2lineup[team_id]
            L_p_id = np.unique(df_game.query('team_id==@team_id')['player_id'])
            

            d_fs4gm[f'{prefix}_id'] = team_id
            d_fs4gm[f'{prefix}_location'] = df_game.query('team_id==@team_id')['team_location'].iloc[0]
            d_fs4gm[f'{prefix}_lineup'] = lineup
            for i, p_id in enumerate(L_p_id):
                d_fs4gm[f'{prefix}_player{i+1}_id'] = p_id
                d_fs4gm[f'{prefix}_player{i+1}_nationality'] = df_game.query('player_id==@p_id')['player_nationality'].iloc[0]
                d_fs4gm[f'{prefix}_player{i+1}_hometown'] = df_game.query('player_id==@p_id')['player_hometown'].iloc[0]
                d_fs4gm[f'{prefix}_player{i+1}_age'] = df_game.query('player_id==@p_id')['player_age'].iloc[0]


            # история игр команды
            df_history4team = df_game_profile_train.query('(timestamp<@timestamp)&(team_id==@team_id)')                    
            
            d_fs4gm.update(df_history4team[L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__all_map_all_start__').add_suffix('__sum').sum().to_dict())
            d_fs4gm.update(df_history4team[L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__all_map_all_start__').add_suffix('__mean').mean().to_dict())

            d_fs4gm.update(df_history4team.query('(map_id==@map_id)&(start_ct==@start_ct)')\
                            [L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__current_map_current_start__').add_suffix('__sum').sum().to_dict())
            d_fs4gm.update(df_history4team.query('(map_id==@map_id)&(start_ct==@start_ct)')\
                            [L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__current_map_current_start__').add_suffix('__mean').mean().to_dict())

            for by_key in L_BY_KEYS:
                for by_value, subdf in df_history4team.groupby(by_key):

                    suffix = f'{by_key}_{by_value}'

                    d_fs4gm.update(subdf[L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__all_map_all_start__').add_suffix('__sum').sum().to_dict())
                    d_fs4gm.update(subdf[L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__all_map_all_start__').add_suffix('__mean').mean().to_dict())

                    d_fs4gm.update(subdf.query('(map_id==@map_id)&(start_ct==@start_ct)')\
                                    [L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__current_map_current_start__').add_suffix('__sum').sum().to_dict())
                    d_fs4gm.update(subdf.query('(map_id==@map_id)&(start_ct==@start_ct)')\
                                    [L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__current_map_current_start__').add_suffix('__mean').mean().to_dict())



            
            # история игр игроков команды
            for i, p_id in enumerate(L_p_id):

                suffix = f'player{i+1}'
                
                # история игр игрока
                df_history4player= df_game_profile_train.query('(timestamp<@timestamp)&(player_id==@p_id)')
                
                # в команде
                df_history4player_in_team = df_history4player.query('team_id==@team_id')
                # в других командах
                df_history4player_not_in_team = df_history4player.query('team_id!=@team_id')            

                d_fs4gm.update(df_history4player_in_team[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__in_team__').add_suffix('__mean').mean().to_dict())
                d_fs4gm.update(df_history4player_in_team[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__in_team__').add_suffix('__sum').sum().to_dict())

                d_fs4gm.update(df_history4player_not_in_team[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__not_in_team__').add_suffix('__mean').mean().to_dict())
                d_fs4gm.update(df_history4player_not_in_team[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__not_in_team__').add_suffix('__sum').sum().to_dict())

                for by_key in L_BY_KEYS:
                    for by_value, subdf in df_history4player_in_team.groupby(by_key):                    
                        prefix_v2 = f'{by_key}_{by_value}'
                        d_fs4gm.update(subdf[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__{prefix_v2}__in_team__').add_suffix('__mean').mean().to_dict())
                        d_fs4gm.update(subdf[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__{prefix_v2}__in_team__').add_suffix('__sum').sum().to_dict())

                for by_key in L_BY_KEYS:
                    for by_value, subdf in df_history4player_not_in_team.groupby(by_key):                    
                        prefix_v2 = f'{by_key}_{by_value}'
                        d_fs4gm.update(subdf[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__{prefix_v2}__not_in_team__').add_suffix('__mean').mean().to_dict())
                        d_fs4gm.update(subdf[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__{prefix_v2}__not_in_team__').add_suffix('__sum').sum().to_dict())  
                  

        with open(path_out, 'wb') as f:
            pickle.dump(d_fs4gm, f)
        del d_fs4gm, df_history4player, df_history4player_in_team, df_history4player_not_in_team, subdf, df_history4team

    except:
        pass

 10%|█         | 1138/11339 [48:51<9:26:25,  3.33s/it] 

: 

In [6]:
PATH2DIR_TEST = r'D:\features_test_21052022'
test_in = [int(x.split('.')[0]) for x in os.listdir(PATH2DIR_TEST)]
test_new = list(set(L_GAME_ID_TEST) - set(test_in))
L_GAME_ID_TEST = test_new

for game_id in tqdm.tqdm(L_GAME_ID_TEST[::-1]):

    try:

        # путь до выходного файлла
        path_out = os.path.join(PATH2DIR_TEST, '{}.pickle'.format(int(game_id)))

        # игра
        df_game = df_game_profile_test.query('id==@game_id')

        # таймстемп игры
        timestamp = df_game['timestamp'].iloc[0]
        # карта игры
        map_id = df_game['map_id'].iloc[0]

        
        # признаки для игры
        d_fs4gm = df_game.iloc[0][L_INFO_KEYS].to_dict()

        d_team_id2start_ct = dict(zip(df_game['team_id'], df_game['start_ct']))
        d_team_id2opponent_id = dict(zip(df_game['team_id'], df_game['opponent_id']))
        d_team_id2lineup = dict(zip(df_game['team_id'], df_game['lineup']))
        d_start_ct2prefix= {1:'team1', 0:'team2'}

        # для каждой команды в игре
        for team_id, start_ct in d_team_id2start_ct.items():
            
            prefix = d_start_ct2prefix[start_ct]
            opponent_id = d_team_id2opponent_id[team_id]
            lineup = d_team_id2lineup[team_id]
            L_p_id = np.unique(df_game.query('team_id==@team_id')['player_id'])
            

            d_fs4gm[f'{prefix}_id'] = team_id
            d_fs4gm[f'{prefix}_location'] = df_game.query('team_id==@team_id')['team_location'].iloc[0]
            d_fs4gm[f'{prefix}_lineup'] = lineup
            for i, p_id in enumerate(L_p_id):
                d_fs4gm[f'{prefix}_player{i+1}_id'] = p_id
                d_fs4gm[f'{prefix}_player{i+1}_nationality'] = df_game.query('player_id==@p_id')['player_nationality'].iloc[0]
                d_fs4gm[f'{prefix}_player{i+1}_hometown'] = df_game.query('player_id==@p_id')['player_hometown'].iloc[0]
                d_fs4gm[f'{prefix}_player{i+1}_age'] = df_game.query('player_id==@p_id')['player_age'].iloc[0]


            # история игр команды
            df_history4team = df_game_profile_train.query('(timestamp<@timestamp)&(team_id==@team_id)')
            
            d_fs4gm.update(df_history4team[L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__all_map_all_start__').add_suffix('__sum').sum().to_dict())
            d_fs4gm.update(df_history4team[L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__all_map_all_start__').add_suffix('__mean').mean().to_dict())

            d_fs4gm.update(df_history4team.query('(map_id==@map_id)&(start_ct==@start_ct)')\
                            [L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__current_map_current_start__').add_suffix('__sum').sum().to_dict())
            d_fs4gm.update(df_history4team.query('(map_id==@map_id)&(start_ct==@start_ct)')\
                            [L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__current_map_current_start__').add_suffix('__mean').mean().to_dict())

            for by_key in L_BY_KEYS:
                for by_value, subdf in df_history4team.groupby(by_key):

                    suffix = f'{by_key}_{by_value}'

                    d_fs4gm.update(subdf[L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__all_map_all_start__').add_suffix('__sum').sum().to_dict())
                    d_fs4gm.update(subdf[L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__all_map_all_start__').add_suffix('__mean').mean().to_dict())

                    d_fs4gm.update(subdf.query('(map_id==@map_id)&(start_ct==@start_ct)')\
                                    [L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__current_map_current_start__').add_suffix('__sum').sum().to_dict())
                    d_fs4gm.update(subdf.query('(map_id==@map_id)&(start_ct==@start_ct)')\
                                    [L_TEAM_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__current_map_current_start__').add_suffix('__mean').mean().to_dict())



            
            # история игр игроков команды
            for i, p_id in enumerate(L_p_id):

                suffix = f'player{i+1}'
                
                # история игр игрока
                df_history4player= df_game_profile_train.query('(timestamp<@timestamp)&(player_id==@p_id)')
                
                # в команде
                df_history4player_in_team = df_history4player.query('team_id==@team_id')
                # в других командах
                df_history4player_not_in_team = df_history4player.query('team_id!=@team_id')            

                d_fs4gm.update(df_history4player_in_team[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__in_team__').add_suffix('__mean').mean().to_dict())
                d_fs4gm.update(df_history4player_in_team[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__in_team__').add_suffix('__sum').sum().to_dict())

                d_fs4gm.update(df_history4player_not_in_team[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__not_in_team__').add_suffix('__mean').mean().to_dict())
                d_fs4gm.update(df_history4player_not_in_team[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__not_in_team__').add_suffix('__sum').sum().to_dict())

                for by_key in L_BY_KEYS:
                    for by_value, subdf in df_history4player_in_team.groupby(by_key):                    
                        prefix_v2 = f'{by_key}_{by_value}'
                        d_fs4gm.update(subdf[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__{prefix_v2}__in_team__').add_suffix('__mean').mean().to_dict())
                        d_fs4gm.update(subdf[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__{prefix_v2}__in_team__').add_suffix('__sum').sum().to_dict())

                for by_key in L_BY_KEYS:
                    for by_value, subdf in df_history4player_not_in_team.groupby(by_key):                    
                        prefix_v2 = f'{by_key}_{by_value}'
                        d_fs4gm.update(subdf[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__{prefix_v2}__not_in_team__').add_suffix('__mean').mean().to_dict())
                        d_fs4gm.update(subdf[L_PLAYER_AGG_KEYS].add_prefix(f'{prefix}__{suffix}__{prefix_v2}__not_in_team__').add_suffix('__sum').sum().to_dict())            

        with open(path_out, 'wb') as f:
            pickle.dump(d_fs4gm, f)
        del d_fs4gm, df_history4player, df_history4player_in_team, df_history4player_not_in_team, subdf, df_history4team

    except:
        pass

  2%|▏         | 44/2307 [02:18<2:40:31,  4.26s/it]

In [4]:
PATH2DIR_TRAIN = r'D:\features_train_21052022'
PATH2DIR_TEST = r'D:\features_test_21052022'


ls = os.listdir(PATH2DIR_TEST)
batch_size = 1000
batches = np.array_split(ls, np.int32(np.ceil(len(ls)/batch_size)))

df_features_test = pd.DataFrame()
for batch in tqdm.tqdm(batches):

    l = []
    for fnm in batch:

        pth = os.path.join(PATH2DIR_TEST, fnm)

        with open(pth, 'rb') as f:
            d_fs4gm = pickle.load(f)
        l.append(d_fs4gm)
        del d_fs4gm
        
    df = pd.DataFrame.from_records(l).fillna(FILL_VALUE).apply(reduce_mem_usage)
    df_features_test = df_features_test.append(df)
    del df




ls = os.listdir(PATH2DIR_TRAIN)
batch_size = 1000
batches = np.array_split(ls, np.int32(np.ceil(len(ls)/batch_size)))

df_features_train = pd.DataFrame()
for batch in tqdm.tqdm(batches):

    l = []
    for fnm in batch:

        pth = os.path.join(PATH2DIR_TRAIN, fnm)

        with open(pth, 'rb') as f:
            d_fs4gm = pickle.load(f)
        l.append(d_fs4gm)
        del d_fs4gm

    df = pd.DataFrame.from_records(l).fillna(FILL_VALUE).apply(reduce_mem_usage)
    df_features_train = df_features_train.append(df)
    del df

100%|██████████| 26/26 [15:48<00:00, 36.48s/it]


In [5]:
feat2use = np.intersect1d(df_features_test.columns, df_features_train.columns)

df_features_train = df_features_train[feat2use]
df_features_test = df_features_test[feat2use]

nan_rate = df_features_train.isna().mean()
key2use = nan_rate[nan_rate<.9].index
df_features_train = df_features_train[key2use]
df_features_test = df_features_test[key2use]

In [6]:
df_features_train.head()

,day,hour,id,league_id,map_id,month,serie_id,serie_tier,team1__all_map_all_start__adr__mean,team1__all_map_all_start__adr__sum,...,team2_player5_hometown,team2_player5_id,team2_player5_nationality,timestamp,tournament_id,tournament_name,tournament_pp_curr,tournament_pp_size,weekday,year
0,1,11,10000,4256,7,11,1932,-999,69.50000,1390.199951,...,Indonesia,24487,ID,1.572607e+09,2748,group a,unitedstatesdollar,6000.0,4,2019
1,2,5,10007,4256,2,11,1932,-999,79.68750,2789.300049,...,Russia,24477,RU,1.572671e+09,2748,group a,unitedstatesdollar,6000.0,5,2019
2,2,7,10012,4256,1,11,1932,-999,61.78125,2471.100098,...,Indonesia,24487,ID,1.572680e+09,2748,group a,unitedstatesdollar,6000.0,5,2019
3,1,6,10017,4256,2,11,1932,-999,78.75000,393.600006,...,Taiwan,24482,TW,1.572588e+09,2749,group b,unitedstatesdollar,6000.0,4,2019
4,1,5,10019,4256,2,11,1932,-999,75.56250,41178.601562,...,Philippines,24457,PH,1.572586e+09,2749,group b,unitedstatesdollar,6000.0,4,2019


In [7]:
df_features_train['tournament_pp_size'] //= 1000
df_features_test['tournament_pp_size'] //= 1000

age_cs=df_features_train.columns[df_features_train.columns.str.contains('age')]
df_features_train= pd.concat([
    df_features_train,
    df_features_train[age_cs].apply(lambda x:x//10).add_suffix('__v1'),
    df_features_train[age_cs].apply(lambda x:x%10).add_suffix('__v2')
],1)

df_features_test= pd.concat([
    df_features_test,
    df_features_test[age_cs].apply(lambda x:x//10).add_suffix('__v1'),
    df_features_test[age_cs].apply(lambda x:x%10).add_suffix('__v2')
],1)

In [8]:
cat_features= [
    
    'year', 'month', 'day', 'weekday', 'hour', 'map_id',
    'league_id', 'serie_id', 'tournament_id', 'serie_tier',
    'tournament_name', 'tournament_pp_size', 'tournament_pp_curr',

    'team1_id', 'team2_id', 'team1_location', 'team2_location', 'team1_lineup', 'team2_lineup',
    'team2_player1_id', 'team2_player2_id', 'team2_player3_id', 'team2_player4_id', 'team2_player5_id',
    'team1_player1_id', 'team1_player2_id', 'team1_player3_id', 'team1_player4_id', 'team1_player5_id',

    'team1_player1_nationality',
    'team1_player1_hometown',
    'team1_player1_age',
    'team1_player1_age__v1',
    'team1_player1_age__v2',
    'team1_player2_nationality',
    'team1_player2_hometown',
    'team1_player2_age',
    'team1_player2_age__v1',
    'team1_player2_age__v2',
    'team1_player3_nationality',
    'team1_player3_hometown',
    'team1_player3_age',
    'team1_player3_age__v1',
    'team1_player3_age__v2',
    'team1_player4_nationality',
    'team1_player4_hometown',
    'team1_player4_age',
    'team1_player4_age__v1',
    'team1_player4_age__v2',
    'team1_player5_nationality',
    'team1_player5_hometown',
    'team1_player5_age',
    'team1_player5_age__v1',
    'team1_player5_age__v2',
    'team2_player1_nationality',
    'team2_player1_hometown',
    'team2_player1_age',
    'team2_player1_age__v1',
    'team2_player1_age__v2',
    'team2_player2_nationality',
    'team2_player2_hometown',
    'team2_player2_age',
    'team2_player2_age__v1',
    'team2_player2_age__v2',
    'team2_player3_nationality',
    'team2_player3_hometown',
    'team2_player3_age',
    'team2_player3_age__v1',
    'team2_player3_age__v2',
    'team2_player4_nationality',
    'team2_player4_hometown',
    'team2_player4_age',
    'team2_player4_age__v1',
    'team2_player4_age__v2',
    'team2_player5_nationality',
    'team2_player5_hometown',
    'team2_player5_age',
    'team2_player5_age__v1',
    'team2_player5_age__v2'
    ]


In [9]:
df_features_train[cat_features] = df_features_train[cat_features].fillna('default').astype('str').astype('category')
df_features_test[cat_features] = df_features_test[cat_features].fillna('default').astype('str').astype('category')

df_features_train = df_features_train.sort_values('id')
df_features_test = df_features_test.sort_values('id')



idx_train = np.intersect1d(df_features_train['id'], df_game_profile['id'])
idx_test = np.intersect1d(df_features_test['id'], df_game_profile['id'])

df_features_train = df_features_train[df_features_train['id'].isin(idx_train)]
df_features_test = df_features_test[df_features_test['id'].isin(idx_test)]

df_features_train = df_features_train.set_index('id')
df_features_test = df_features_test.set_index('id')

for game_id in tqdm.tqdm(df_features_train.index):
    df_game = df_game_profile.query('id==@game_id')    
    df_features_train.loc[game_id, 'team1_win'] = int((df_game.query('start_ct==1')['win']==1).all())

for game_id in tqdm.tqdm(df_features_test.index):
    df_game = df_game_profile.query('id==@game_id')    
    df_features_test.loc[game_id, 'team1_win'] = int((df_game.query('start_ct==1')['win']==1).all())

100%|██████████| 1665/1665 [00:14<00:00, 112.75it/s]


In [10]:
X_train, X_valid, y_train, y_valid = train_test_split(
    df_features_train.drop('team1_win', 1),
    df_features_train['team1_win'].astype(int),
    test_size = .2,
    shuffle=False
)
X_test, y_test = df_features_test.drop('team1_win', 1), df_features_test['team1_win'].astype(int)

In [11]:
X_train.shape

(20104, 49912)

### 5. отбор признаков

In [12]:
d_sel_res = select_features(X_train, X_valid, X_test, y_train, y_valid, y_test)

ITER#1/50
Learning rate set to 0.0665
0:	learn: 0.6917071	test: 0.6917599	best: 0.6917599 (0)	total: 311ms	remaining: 5m 10s
1:	learn: 0.6904470	test: 0.6905156	best: 0.6905156 (1)	total: 427ms	remaining: 3m 32s
2:	learn: 0.6893241	test: 0.6896063	best: 0.6896063 (2)	total: 552ms	remaining: 3m 3s
3:	learn: 0.6882630	test: 0.6885465	best: 0.6885465 (3)	total: 690ms	remaining: 2m 51s
4:	learn: 0.6872207	test: 0.6874113	best: 0.6874113 (4)	total: 795ms	remaining: 2m 38s
5:	learn: 0.6863314	test: 0.6867502	best: 0.6867502 (5)	total: 907ms	remaining: 2m 30s
6:	learn: 0.6856215	test: 0.6860713	best: 0.6860713 (6)	total: 1.02s	remaining: 2m 24s
7:	learn: 0.6848122	test: 0.6851536	best: 0.6851536 (7)	total: 1.14s	remaining: 2m 20s
8:	learn: 0.6841236	test: 0.6845870	best: 0.6845870 (8)	total: 1.24s	remaining: 2m 16s
9:	learn: 0.6834332	test: 0.6841916	best: 0.6841916 (9)	total: 1.35s	remaining: 2m 14s
10:	learn: 0.6828698	test: 0.6835292	best: 0.6835292 (10)	total: 1.46s	remaining: 2m 11s
11:	

100%|██████████| 10/10 [00:51<00:00,  5.13s/it]


> step#2, iter#2, selected features counter: 76


100%|██████████| 10/10 [00:20<00:00,  2.04s/it]


> step#2, iter#3, selected features counter: 69


100%|██████████| 10/10 [00:17<00:00,  1.71s/it]


> step#2, iter#4, selected features counter: 67


100%|██████████| 10/10 [00:16<00:00,  1.61s/it]


Learning rate set to 0.0665
0:	learn: 0.6895957	test: 0.6897321	best: 0.6897321 (0)	total: 44.3ms	remaining: 44.2s
1:	learn: 0.6858150	test: 0.6857010	best: 0.6857010 (1)	total: 92.9ms	remaining: 46.3s
2:	learn: 0.6825732	test: 0.6830777	best: 0.6830777 (2)	total: 142ms	remaining: 47s
3:	learn: 0.6796880	test: 0.6797284	best: 0.6797284 (3)	total: 197ms	remaining: 49.1s
4:	learn: 0.6768478	test: 0.6771084	best: 0.6771084 (4)	total: 263ms	remaining: 52.3s
5:	learn: 0.6741742	test: 0.6746833	best: 0.6746833 (5)	total: 317ms	remaining: 52.5s
6:	learn: 0.6718351	test: 0.6724659	best: 0.6724659 (6)	total: 370ms	remaining: 52.5s
7:	learn: 0.6692820	test: 0.6704229	best: 0.6704229 (7)	total: 430ms	remaining: 53.3s
8:	learn: 0.6666938	test: 0.6681260	best: 0.6681260 (8)	total: 491ms	remaining: 54.1s
9:	learn: 0.6647317	test: 0.6665879	best: 0.6665879 (9)	total: 543ms	remaining: 53.7s
10:	learn: 0.6630537	test: 0.6652926	best: 0.6652926 (10)	total: 602ms	remaining: 54.1s
11:	learn: 0.6612093	tes

In [21]:
for i, feature in enumerate(d_sel_res['features']):
    print('{}. {}'.format(i+1, feature))
print('auc = {:.2f}'.format(d_sel_res['score']))

1. league_id
2. team1__all_map_all_start__first_kills_diff__mean
3. team1__all_map_all_start__first_kills_diff__sum
4. team1__all_map_all_start__h1_win_count__mean
5. team1__all_map_all_start__k_d_diff__mean
6. team1__all_map_all_start__k_d_diff__sum
7. team1__all_map_all_start__win__mean
8. team1__current_map_current_start__adr__mean
9. team1__current_map_current_start__k_d_diff__sum
10. team1__day_5__all_map_all_start__win__mean
11. team1__day_5__current_map_current_start__k_d_diff__mean
12. team1__hour_0__all_map_all_start__win__mean
13. team1__month_11__current_map_current_start__kast__mean
14. team1__month_6__current_map_current_start__assists__mean
15. team1__player1__not_in_team__k_d_diff__mean
16. team1__player1__serie_tier_s__in_team__headshots__mean
17. team1__player1__serie_tier_s__in_team__rating__sum
18. team1__player1__weekday_3__in_team__rating__mean
19. team1__player1__weekday_4__not_in_team__headshots__mean
20. team1__player1__year_2018__not_in_team__kills__sum
21. tea

In [18]:
X_train = X_train[d_sel_res['features']]
X_valid = X_valid[d_sel_res['features']]
X_test = X_test[d_sel_res['features']]



In [19]:
params = CONST_PARAMS.copy()
params['cat_features'] = np.where(X_train.dtypes=='category')[0]
params['use_best_model'] = True
params['random_state'] = SEED

model = cb.CatBoostClassifier(**params)      
model.fit(X_train, y_train,  eval_set=(X_valid, y_valid), early_stopping_rounds=EARLY_STOPPING)
te_proba = model.predict_proba(X_test)[:, 1]



Learning rate set to 0.0665
0:	learn: 0.6895957	test: 0.6897321	best: 0.6897321 (0)	total: 46.2ms	remaining: 46.1s
1:	learn: 0.6858150	test: 0.6857010	best: 0.6857010 (1)	total: 97.2ms	remaining: 48.5s
2:	learn: 0.6825732	test: 0.6830777	best: 0.6830777 (2)	total: 148ms	remaining: 49.3s
3:	learn: 0.6796880	test: 0.6797284	best: 0.6797284 (3)	total: 209ms	remaining: 52s
4:	learn: 0.6768478	test: 0.6771084	best: 0.6771084 (4)	total: 290ms	remaining: 57.8s
5:	learn: 0.6741742	test: 0.6746833	best: 0.6746833 (5)	total: 367ms	remaining: 1m
6:	learn: 0.6718351	test: 0.6724659	best: 0.6724659 (6)	total: 419ms	remaining: 59.5s
7:	learn: 0.6692820	test: 0.6704229	best: 0.6704229 (7)	total: 477ms	remaining: 59.1s
8:	learn: 0.6666938	test: 0.6681260	best: 0.6681260 (8)	total: 531ms	remaining: 58.4s
9:	learn: 0.6647317	test: 0.6665879	best: 0.6665879 (9)	total: 598ms	remaining: 59.2s
10:	learn: 0.6630537	test: 0.6652926	best: 0.6652926 (10)	total: 674ms	remaining: 1m
11:	learn: 0.6612093	test: 0.6

In [22]:
proba = te_proba
tresholds = np.unique(proba)
best_acc = -np.inf
for treshold in tresholds:
    accuracy = (y_test==(proba>=treshold).astype(int)).mean()
    if accuracy>best_acc:
        best_acc = accuracy

In [24]:
PATH_TO_MODELS = r'C:\Users\Sergey\anaconda3\Scripts\Models'

In [25]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Flatten
from tensorflow.keras.models import Model

In [74]:
EMB_DIM = 64
PLAYERS_COUNT = 4589
TEAMS_COUNT = 1330

p1_input = Input(shape = 1, name = 'p1')
p1_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p1_embedding')(p1_input)
p1_vec = Flatten(name='Fp1')(p1_emb)
p1_model = Model(inputs = p1_input, outputs = p1_vec)

p2_input = Input(shape = 1, name = 'p2')
p2_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p2_embedding')(p2_input)
p2_vec = Flatten(name='Fp2')(p2_emb)
p2_model = Model(inputs = p2_input, outputs = p2_vec)

p3_input = Input(shape = 1, name = 'p3')
p3_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p3_embedding')(p3_input)
p3_vec = Flatten(name='Fp3')(p3_emb)
p3_model = Model(inputs = p3_input, outputs = p3_vec)

p4_input = Input(shape = 1, name = 'p4')
p4_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p4_embedding')(p4_input)
p4_vec = Flatten(name='Fp4')(p4_emb)
p4_model = Model(inputs = p4_input, outputs = p4_vec)

p5_input = Input(shape = 1, name = 'p5')
p5_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p5_embedding')(p5_input)
p5_vec = Flatten(name='Fp5')(p5_emb)
p5_model = Model(inputs = p5_input, outputs = p5_vec)

p6_input = Input(shape = 1, name = 'p6')
p6_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p6_embedding')(p6_input)
p6_vec = Flatten(name='Fp6')(p6_emb)
p6_model = Model(inputs = p6_input, outputs = p6_vec)

p7_input = Input(shape = 1, name = 'p7')
p7_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p7_embedding')(p7_input)
p7_vec = Flatten(name='Fp7')(p7_emb)
p7_model = Model(inputs = p7_input, outputs = p7_vec)

p8_input = Input(shape = 1, name = 'p8')
p8_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p8_embedding')(p8_input)
p8_vec = Flatten(name='Fp8')(p8_emb)
p8_model = Model(inputs = p8_input, outputs = p8_vec)

p9_input = Input(shape = 1, name = 'p9')
p9_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p9_embedding')(p9_input)
p9_vec = Flatten(name='Fp9')(p9_emb)
p9_model = Model(inputs = p9_input, outputs = p9_vec)

p10_input = Input(shape = 1, name = 'p10')
p10_emb = Embedding(output_dim = EMB_DIM, input_dim = PLAYERS_COUNT, input_length = 1, name = 'p10_embedding')(p10_input)
p10_vec = Flatten(name='Fp10')(p10_emb)
p10_model = Model(inputs = p10_input, outputs = p10_vec)

t1_input = Input(shape = 1, name = 't1')
t1_emb = Embedding(output_dim = EMB_DIM, input_dim = TEAMS_COUNT, input_length = 1, name = 't1_embedding')(t1_input)
t1_vec = Flatten(name = 'Ft1')(t1_emb)
t1_model = Model(inputs = t1_input, outputs = t1_vec)

t2_input = Input(shape = 1, name = 't2')
t2_emb = Embedding(output_dim = EMB_DIM, input_dim = TEAMS_COUNT, input_length = 1, name = 't2_embedding')(t2_input)
t2_vec = Flatten(name = 'Ft2')(t2_emb)
t2_model = Model(inputs = t2_input, outputs = t2_vec)

In [75]:
# Load weigts
p1_model.load_weights(os.path.join(PATH_TO_MODELS, 'p1_model.h5'))
p2_model.load_weights(os.path.join(PATH_TO_MODELS, 'p2_model.h5'))
p3_model.load_weights(os.path.join(PATH_TO_MODELS, 'p3_model.h5'))
p4_model.load_weights(os.path.join(PATH_TO_MODELS, 'p4_model.h5'))
p5_model.load_weights(os.path.join(PATH_TO_MODELS, 'p5_model.h5'))
p6_model.load_weights(os.path.join(PATH_TO_MODELS, 'p6_model.h5'))
p7_model.load_weights(os.path.join(PATH_TO_MODELS, 'p7_model.h5'))
p8_model.load_weights(os.path.join(PATH_TO_MODELS, 'p8_model.h5'))
p9_model.load_weights(os.path.join(PATH_TO_MODELS, 'p9_model.h5'))
p10_model.load_weights(os.path.join(PATH_TO_MODELS, 'p10_model.h5'))
t1_model.load_weights(os.path.join(PATH_TO_MODELS, 't1_model.h5'))
t2_model.load_weights(os.path.join(PATH_TO_MODELS, 't2_model.h5'))

In [76]:
zipped = zip([p1_model, p2_model, p3_model, p4_model, p5_model, p6_model, p7_model, p8_model, p9_model, p10_model, t1_model, t2_model],
['player1_id', 'player2_id', 'player3_id', 'player4_id', 'player5_id', 'player6_id', 'player7_id', 'player8_id', 'player9_id', 'player10_id', 'team1_id','team2_id'])

In [72]:
with open('df_game_profile_train.pickle', 'rb') as f:
    df_game_profile_train = pickle.load(f)

with open('df_game_profile_test.pickle', 'rb') as f:
    df_game_profile_test = pickle.load(f)


In [73]:
df_embed_train = pd.DataFrame()
for model, key in tqdm.tqdm(zipped):
    embed = model.predict(df_game_profile_train[key].values)
    df = pd.DataFrame(embed)
    df_embed_train = pd.concat([df_embed_train, df], 1)
    del df

12it [00:06,  1.93it/s]


In [77]:
df_embed_test = pd.DataFrame()
for model, key in tqdm.tqdm(zipped):
    embed = model.predict(df_game_profile_test[key].values)
    df = pd.DataFrame(embed)
    df_embed_test = pd.concat([df_embed_test, df], 1)
    del df

12it [00:01,  7.96it/s]


Int64Index([    2,     3,     8,    25,    33,    37,    39,    40,    49,
               53,
            ...
            53350, 53351, 53352, 53353, 53355, 53356, 53357, 53358, 53359,
            53360],
           dtype='int64', name='id', length=20104)

In [83]:
df_embed_train.index = df_game_profile_train['id']
df_embed_test.index = df_game_profile_test['id']


In [90]:
X_train = pd.concat([X_train, df_embed_train.loc[X_train.index]], 1)
X_valid = pd.concat([X_valid, df_embed_train.loc[X_valid.index]], 1)
X_test = pd.concat([X_test, df_embed_test.loc[X_test.index]], 1)

In [92]:
X_train.columns = [f'feature{i}' for i in range(X_train.shape[1])]
X_valid.columns = [f'feature{i}' for i in range(X_valid.shape[1])]
X_test.columns = [f'feature{i}' for i in range(X_test.shape[1])]

In [97]:
X_train.to_pickle('X_train.pickle')

In [93]:
d_sel_res = select_features(X_train, X_valid, X_test, y_train, y_valid, y_test)

ITER#1/1
Learning rate set to 0.0665
0:	learn: 0.6896896	test: 0.6898655	best: 0.6898655 (0)	total: 128ms	remaining: 2m 7s
1:	learn: 0.6862419	test: 0.6871565	best: 0.6871565 (1)	total: 257ms	remaining: 2m 8s
2:	learn: 0.6832586	test: 0.6842764	best: 0.6842764 (2)	total: 383ms	remaining: 2m 7s
3:	learn: 0.6804467	test: 0.6817998	best: 0.6817998 (3)	total: 497ms	remaining: 2m 3s
4:	learn: 0.6776471	test: 0.6791127	best: 0.6791127 (4)	total: 622ms	remaining: 2m 3s
5:	learn: 0.6750746	test: 0.6769742	best: 0.6769742 (5)	total: 743ms	remaining: 2m 3s
6:	learn: 0.6728114	test: 0.6749231	best: 0.6749231 (6)	total: 874ms	remaining: 2m 4s
7:	learn: 0.6704977	test: 0.6729545	best: 0.6729545 (7)	total: 1.01s	remaining: 2m 5s
8:	learn: 0.6679237	test: 0.6707161	best: 0.6707161 (8)	total: 1.15s	remaining: 2m 6s
9:	learn: 0.6658138	test: 0.6692714	best: 0.6692714 (9)	total: 1.29s	remaining: 2m 7s
10:	learn: 0.6636614	test: 0.6671006	best: 0.6671006 (10)	total: 1.43s	remaining: 2m 8s
11:	learn: 0.66

100%|██████████| 10/10 [00:55<00:00,  5.52s/it]


Learning rate set to 0.0665
0:	learn: 0.6897160	test: 0.6897394	best: 0.6897394 (0)	total: 53.2ms	remaining: 53.2s
1:	learn: 0.6864758	test: 0.6866664	best: 0.6866664 (1)	total: 105ms	remaining: 52.5s
2:	learn: 0.6827095	test: 0.6827187	best: 0.6827187 (2)	total: 159ms	remaining: 53s
3:	learn: 0.6796434	test: 0.6799088	best: 0.6799088 (3)	total: 214ms	remaining: 53.4s
4:	learn: 0.6768920	test: 0.6772950	best: 0.6772950 (4)	total: 274ms	remaining: 54.5s
5:	learn: 0.6745955	test: 0.6751368	best: 0.6751368 (5)	total: 341ms	remaining: 56.5s
6:	learn: 0.6721562	test: 0.6730728	best: 0.6730728 (6)	total: 401ms	remaining: 56.9s
7:	learn: 0.6698495	test: 0.6709090	best: 0.6709090 (7)	total: 470ms	remaining: 58.3s
8:	learn: 0.6678550	test: 0.6690737	best: 0.6690737 (8)	total: 528ms	remaining: 58.1s
9:	learn: 0.6658986	test: 0.6675422	best: 0.6675422 (9)	total: 581ms	remaining: 57.5s
10:	learn: 0.6640022	test: 0.6663528	best: 0.6663528 (10)	total: 640ms	remaining: 57.6s
11:	learn: 0.6621137	test

In [94]:
X_train = X_train[d_sel_res['features']]
X_valid = X_valid[d_sel_res['features']]
X_test = X_test[d_sel_res['features']]

params = CONST_PARAMS.copy()
params['cat_features'] = np.where(X_train.dtypes=='category')[0]
params['use_best_model'] = True
params['random_state'] = SEED

model = cb.CatBoostClassifier(**params)      
model.fit(X_train, y_train,  eval_set=(X_valid, y_valid), early_stopping_rounds=EARLY_STOPPING)
te_proba = model.predict_proba(X_test)[:, 1]

proba = te_proba
tresholds = np.unique(proba)
best_acc = -np.inf
for treshold in tresholds:
    accuracy = (y_test==(proba>=treshold).astype(int)).mean()
    if accuracy>best_acc:
        best_acc = accuracy

Learning rate set to 0.0665
0:	learn: 0.6897160	test: 0.6897394	best: 0.6897394 (0)	total: 54.5ms	remaining: 54.5s
1:	learn: 0.6864758	test: 0.6866664	best: 0.6866664 (1)	total: 109ms	remaining: 54.3s
2:	learn: 0.6827095	test: 0.6827187	best: 0.6827187 (2)	total: 163ms	remaining: 54.3s
3:	learn: 0.6796434	test: 0.6799088	best: 0.6799088 (3)	total: 222ms	remaining: 55.4s
4:	learn: 0.6768920	test: 0.6772950	best: 0.6772950 (4)	total: 287ms	remaining: 57.1s
5:	learn: 0.6745955	test: 0.6751368	best: 0.6751368 (5)	total: 365ms	remaining: 1m
6:	learn: 0.6721562	test: 0.6730728	best: 0.6730728 (6)	total: 419ms	remaining: 59.4s
7:	learn: 0.6698495	test: 0.6709090	best: 0.6709090 (7)	total: 479ms	remaining: 59.4s
8:	learn: 0.6678550	test: 0.6690737	best: 0.6690737 (8)	total: 542ms	remaining: 59.7s
9:	learn: 0.6658986	test: 0.6675422	best: 0.6675422 (9)	total: 602ms	remaining: 59.6s
10:	learn: 0.6640022	test: 0.6663528	best: 0.6663528 (10)	total: 686ms	remaining: 1m 1s
11:	learn: 0.6621137	test:

In [95]:
best_acc

0.6222222222222222